In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [3]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, LayerNormalization, Reshape
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

2025-08-09 14:38:41.821662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754750321.988200      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754750322.038995      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
df_train = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv')
df_test = pd.read_csv('/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv')

In [6]:
df_train.columns = ['ID', 'Entity', 'Sentiment', 'Text']
df_test.columns = ['ID', 'Entity', 'Sentiment', 'Text']

In [7]:
df_train.isnull().sum()

ID             0
Entity         0
Sentiment      0
Text         686
dtype: int64

In [8]:
df_test.isnull().sum()

ID           0
Entity       0
Sentiment    0
Text         0
dtype: int64

In [9]:
df_train.dropna(subset=['Text'], inplace=True)
df_test.dropna(subset=['Text'], inplace=True)

In [10]:
print("Training data shape:", df_train.shape)
print("Test data shape:", df_test.shape)
print("\nTraining data columns:", df_train.columns.tolist())

Training data shape: (73995, 4)
Test data shape: (999, 4)

Training data columns: ['ID', 'Entity', 'Sentiment', 'Text']


In [11]:
df_train.head()

,ID,Entity,Sentiment,Text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [12]:
df_train['Entity'].unique()

array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)', 'NBA2K', 'Dota2', 'PlayStation5(PS5)',
       'WorldOfCraft', 'CS-GO', 'Google', 'AssassinsCreed', 'ApexLegends',
       'LeagueOfLegends', 'Fortnite', 'Microsoft', 'Hearthstone',
       'Battlefield', 'PlayerUnknownsBattlegrounds(PUBG)', 'Verizon',
       'HomeDepot', 'FIFA', 'RedDeadRedemption(RDR)', 'CallOfDuty',
       'TomClancysRainbowSix', 'Facebook', 'GrandTheftAuto(GTA)',
       'MaddenNFL', 'johnson&johnson', 'Cyberpunk2077',
       'TomClancysGhostRecon', 'Nvidia'], dtype=object)

In [13]:
df_train['Sentiment'].unique()

array(['Positive', 'Neutral', 'Negative', 'Irrelevant'], dtype=object)

In [14]:
# Create label encoding mapping from training data
unique_sentiments = df_train['Sentiment'].unique()
sentiment_to_label = {sentiment: i for i, sentiment in enumerate(unique_sentiments)}
label_to_sentiment = {i: sentiment for i, sentiment in enumerate(unique_sentiments)}

In [15]:
for sentiment, label_id in sentiment_to_label.items():
    print(f"{sentiment}: {label_id}")

Positive: 0
Neutral: 1
Negative: 2
Irrelevant: 3


In [16]:
df_train['label'] = df_train['Sentiment'].map(sentiment_to_label)
df_test['label'] = df_test['Sentiment'].map(sentiment_to_label)

In [17]:
df_train.head()

,ID,Entity,Sentiment,Text,label
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,0
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,0
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,0
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,0
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...,0


In [18]:
# Check if all labels were mapped successfully
print("Train - Missing labels after mapping:", df_train['label'].isnull().sum())
print("Test - Missing labels after mapping:", df_test['label'].isnull().sum())

Train - Missing labels after mapping: 0
Test - Missing labels after mapping: 0


## Text Cleaning

In [19]:
def clean_text(text):
    """
    Cleans text by lowercasing, removing stopwords, and applying stemming.
    Also handles Twitter-specific elements like mentions, hashtags, URLs.
    """
    if pd.isna(text):  # Handle missing values
        return ""

    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))

    # Convert to string if not already
    text = str(text)

    # Convert to lowercase
    text = text.lower()

    # Remove Twitter-specific elements
    text = re.sub(r'@\w+', '', text)  # Remove mentions (@username)
    text = re.sub(r'#\w+', '', text)  # Remove hashtags (#hashtag)
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)  # Remove URLs
    text = re.sub(r'rt\s+', '', text)  # Remove "RT" (retweet)

    # Remove non-alphabetic characters (keep spaces)
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Tokenize and remove stopwords, apply stemming
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stop_words and len(word) > 1]

    # Join words back into a string
    return " ".join(words)

In [20]:
# Apply text cleaning to both datasets
df_train['cleaned_text'] = df_train['Text'].apply(clean_text)

df_test['cleaned_text'] = df_test['Text'].apply(clean_text)

In [21]:
df_train.head()

,ID,Entity,Sentiment,Text,label,cleaned_text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,0,come border kill
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,0,im get borderland kill
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...,0,im come borderland murder
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...,0,im get borderland murder
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...,0,im get borderland murder


In [22]:
# Filter out empty texts
df_train = df_train[df_train['cleaned_text'].str.len() > 0]
df_test = df_test[df_test['cleaned_text'].str.len() > 0]

In [23]:
print(f"After cleaning - Train: {len(df_train)}, Test: {len(df_test)}")

After cleaning - Train: 72240, Test: 998


In [24]:
# Display examples of original vs cleaned text
print("\nExamples of text cleaning:")
print("="*80)
for i in range(3):
    print(f"Original: {df_train.iloc[i]['Text']}")
    print(f"Cleaned:  {df_train.iloc[i]['cleaned_text']}")
    print(f"Sentiment: {df_train.iloc[i]['Sentiment']}")
    print("-"*80)


Examples of text cleaning:
Original: I am coming to the borders and I will kill you all,
Cleaned:  come border kill
Sentiment: Positive
--------------------------------------------------------------------------------
Original: im getting on borderlands and i will kill you all,
Cleaned:  im get borderland kill
Sentiment: Positive
--------------------------------------------------------------------------------
Original: im coming on borderlands and i will murder you all,
Cleaned:  im come borderland murder
Sentiment: Positive
--------------------------------------------------------------------------------


## TF-IDF Vectorization

In [25]:
# TF-IDF Vectorization parameters
max_features = 5000  # Maximum number of features (vocabulary size)
max_df = 0.95        # Ignore words that appear in more than 95% of documents
min_df = 2           # Ignore words that appear in fewer than 2 documents

In [26]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_features=max_features,
    max_df=max_df,
    min_df=min_df,
    ngram_range=(1, 2),  # Use both unigrams and bigrams
    lowercase=True,      # Already done in cleaning but just to be safe
    stop_words='english' # Additional stopword removal
)

In [27]:
# Fit and transform training data
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['cleaned_text'])

# Transform test data (don't fit on test data to avoid data leakage)
X_test_tfidf = tfidf_vectorizer.transform(df_test['cleaned_text'])

In [28]:
# Convert sparse matrices to dense arrays for LSTM
X_train = X_train_tfidf.toarray()
X_test = X_test_tfidf.toarray()

In [29]:
# Prepare labels
y_train = df_train['label'].values
y_test = df_test['label'].values

In [30]:
# Display shapes and info
print(f"\nData shapes after TF-IDF:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")


Data shapes after TF-IDF:
X_train shape: (72240, 5000)
X_test shape: (998, 5000)
y_train shape: (72240,)
y_test shape: (998,)


## Training LSTM

In [31]:
# Get data info
num_features = X_train.shape[1]  # TF-IDF vector size
num_classes = len(np.unique(y_train))  # Number of sentiment classes

In [32]:
print(f"Number of features (TF-IDF): {num_features}")
print(f"Number of classes: {num_classes}")
print(f"Classes: {list(label_to_sentiment.values())}")

Number of features (TF-IDF): 5000
Number of classes: 4
Classes: ['Positive', 'Neutral', 'Negative', 'Irrelevant']


In [33]:
# Convert labels to categorical (one-hot encoding)
y_train_categorical = to_categorical(y_train, num_classes=num_classes)
y_test_categorical = to_categorical(y_test, num_classes=num_classes)

In [34]:
print(f"y_train_categorical shape: {y_train_categorical.shape}")

y_train_categorical shape: (72240, 4)


In [35]:
# Split training data for validation
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train, y_train_categorical, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Train split: {X_train_split.shape}, Validation split: {X_val_split.shape}")

Train split: (57792, 5000), Validation split: (14448, 5000)


In [36]:
sequence_length = 1  # Single timestep
X_train_reshaped = X_train_split.reshape(X_train_split.shape[0], sequence_length, num_features)
X_val_reshaped = X_val_split.reshape(X_val_split.shape[0], sequence_length, num_features)
X_test_reshaped = X_test.reshape(X_test.shape[0], sequence_length, num_features)

In [37]:
print(f"Reshaped data: {X_train_reshaped.shape}")

Reshaped data: (57792, 1, 5000)


In [38]:
# Define LSTM model for TF-IDF input
model = Sequential([
    # Input layer - no embedding needed for TF-IDF
    LSTM(256, return_sequences=True, input_shape=(sequence_length, num_features),
         recurrent_dropout=0.2, dropout=0.2),
    LayerNormalization(),

    Bidirectional(LSTM(128, return_sequences=True, recurrent_dropout=0.2, dropout=0.2)),
    LayerNormalization(),
    Dropout(0.3),

    Bidirectional(LSTM(64, recurrent_dropout=0.2, dropout=0.2)),
    LayerNormalization(),
    Dropout(0.4),

    # Dense layers
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.3),

    # Output layer for multiclass classification
    Dense(num_classes, activation='softmax')
])

I0000 00:00:1754750360.713829      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1754750360.714582      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [39]:
# Compile model
model.compile(
    loss='categorical_crossentropy',  # For multiclass classification
    optimizer=Adam(learning_rate=1e-3),
    metrics=['accuracy', 'precision', 'recall']
)

In [40]:
# Display model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 256)         │     5,383,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 1, 256)         │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 1, 256)         │       394,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 1, 256)         │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 256)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 128)            │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 128)            │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,968,068 (22.77 MB)

 Trainable params: 5,968,068 (22.77 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
# Define callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=7,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        verbose=1
    )
]

In [42]:
# Train the model
history = model.fit(
    X_train_reshaped, y_train_split,
    validation_data=(X_val_reshaped, y_val_split),
    epochs=50,
    batch_size=32,  # Smaller batch size for better convergence
    callbacks=callbacks,
    verbose=1
)

Epoch 1/50
1806/1806 ━━━━━━━━━━━━━━━━━━━━ 71s 28ms/step - accuracy: 0.3521 - loss: 1.3903 - precision: 0.4759 - recall: 0.1008 - val_accuracy: 0.5696 - val_loss: 1.0564 - val_precision: 0.7912 - val_recall: 0.3101 - learning_rate: 0.0010
Epoch 2/50
1806/1806 ━━━━━━━━━━━━━━━━━━━━ 46s 25ms/step - accuracy: 0.5711 - loss: 1.0570 - precision: 0.7351 - recall: 0.3713 - val_accuracy: 0.6417 - val_loss: 0.9210 - val_precision: 0.8023 - val_recall: 0.4176 - learning_rate: 0.0010
Epoch 3/50
1806/1806 ━━━━━━━━━━━━━━━━━━━━ 45s 25ms/step - accuracy: 0.6305 - loss: 0.9462 - precision: 0.7630 - recall: 0.4615 - val_accuracy: 0.6706 - val_loss: 0.8315 - val_precision: 0.8181 - val_recall: 0.4844 - learning_rate: 0.0010
Epoch 4/50
1806/1806 ━━━━━━━━━━━━━━━━━━━━ 46s 26ms/step - accuracy: 0.6659 - loss: 0.8571 - precision: 0.7823 - recall: 0.5183 - val_accuracy: 0.6962 - val_loss: 0.7842 - val_precision: 0.8411 - val_recall: 0.5041 - learning_rate: 0.0010
Epoch 5/50
1806/1806 ━━━━━━━━━━━━━━━━━━━━ 45s 25

In [43]:
import joblib
import pickle

In [44]:
# 1. حفظ النموذج المدرب 
model.save('sentiment_lstm_model.keras')

In [45]:
# 2. حفظ الـ TF-IDF vectorizer 
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [46]:
# 3. حفظ تشفير التسميات 
label_mappings = {
    'sentiment_to_label': sentiment_to_label,
    'label_to_sentiment': label_to_sentiment
}

with open('label_mappings.pkl', 'wb') as f:
    pickle.dump(label_mappings, f)